In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

import pinecone
import openai


load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')
WOLFRAM_ALPHA_APPID = os.getenv('WOLFRAM_ALPHA_APPID')
PROMPTLAYER_API_KEY = os.getenv('PROMPTLAYER_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_ENV')

/Users/vladbordei/anaconda3/envs/oaie/lib/python3.8/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
reviews = pd.read_csv('reviews.csv')

In [4]:
def get_ada_embedding(text):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=[text], model="text-embedding-ada-002")["data"][0]["embedding"]


def get_text_from_embedding(embedding):
    return openai.Embedding.retrieve(embedding, model="text-embedding-ada-002")["data"][0]["text"]


In [5]:
# 
class Singleton(type):
    """
    Singleton metaclass for ensuring only one instance of a class.
    """

    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(
                Singleton, cls).__call__(
                *args, **kwargs)
        return cls._instances[cls]



class PineconeMemory(metaclass=Singleton):
    def __init__(self):
        pinecone_api_key = PINECONE_API_KEY
        pinecone_region = PINECONE_ENV
        pinecone.init(api_key=pinecone_api_key, environment=pinecone_region)
        dimension = 1536
        metric = "cosine"
        pod_type = "p1"
        table_name = "oaie"

        # this assumes we don't start with memory.
        self.vec_num = 0
        if table_name not in pinecone.list_indexes():
            pinecone.create_index(table_name, dimension=dimension, metric=metric, pod_type=pod_type)
        
        # Add this line to initialize the 'index' attribute
        self.index = pinecone.Index(index_name=table_name)




    def add(self, data):
        vector = get_ada_embedding(data)
        resp = self.index.upsert([(str(self.vec_num), vector, {"raw_text": data})])
        _text = f"Inserting data into memory at index: {self.vec_num}:\n data: {data}"
        self.vec_num += 1
        return _text

    def get(self, data):
        return self.get_relevant(data, 1)

    def clear(self):
        self.index.delete(deleteAll=True)
        return "Obliviated"

    def get_relevant(self, data, num_relevant=5):
        """
        Returns all the data in the memory that is relevant to the given data.
        :param data: The data to compare to.
        :param num_relevant: The number of relevant data to return. Defaults to 5
        """
        query_embedding = get_ada_embedding(data)
        results = self.index.query(query_embedding, top_k=num_relevant, include_metadata=True)
        sorted_results = sorted(results.matches, key=lambda x: x.score)
        return [str(item['metadata']["raw_text"]) for item in sorted_results]

    def get_stats(self):
        return self.index.describe_index_stats()

In [6]:
# Initialize memory and make sure it is empty.
# this is particularly important for indexing and referencing pinecone memory
memory = PineconeMemory()
memory.clear()
full_message_history = []

In [7]:
print('Using memory of type: ' + memory.__class__.__name__)

Using memory of type: PineconeMemory


In [ ]:
    memory.add(memory_to_add)


    memory_to_add = f"Assistant Reply: {assistant_reply} " \
                    f"\nResult: {result} " \
                    f"\nHuman Feedback: {user_input} "

In [18]:
memory_to_add = reviews['review'][0]

In [19]:
memory_to_add

'Stuck to 45 year olds inside of ear We tried to remove but child was nervous about us touching his ear Went to ER same night and they pushed it deeper Went to ENT and he could not get it out At this point close to our childs ear drum We ended up having to sedate a young child as advised by specialist to remove Terrible time to have to go to the hospital during high cases of COVID Then we had to wait bc of limited availability So this stone was stuck in his ear for a few days causing bleeding and pain I dont advise unless this is for a teenager and no small kids around\n\nMy daughter requested this for her birthday Honestly its just sparkly 3D stickers With a fancy thing to stamp it on but really you could just peel it off with your finger and stick it anywheres adhesive doesnt last long so these dont stick to clothesfabricshoes etc very long It lasts a day and the You have them all over your house  My daughter enjoyed it though but after she used all the stickies in a couple days  its

In [20]:
memory.add(memory_to_add)

'Inserting data into memory at index: 1:\n data: Stuck to 45 year olds inside of ear We tried to remove but child was nervous about us touching his ear Went to ER same night and they pushed it deeper Went to ENT and he could not get it out At this point close to our childs ear drum We ended up having to sedate a young child as advised by specialist to remove Terrible time to have to go to the hospital during high cases of COVID Then we had to wait bc of limited availability So this stone was stuck in his ear for a few days causing bleeding and pain I dont advise unless this is for a teenager and no small kids around\n\nMy daughter requested this for her birthday Honestly its just sparkly 3D stickers With a fancy thing to stamp it on but really you could just peel it off with your finger and stick it anywheres adhesive doesnt last long so these dont stick to clothesfabricshoes etc very long It lasts a day and the You have them all over your house  My daughter enjoyed it though but after

In [13]:
memory.get(memory_to_add)

['Bought this for my 5 year old daughter She loves it It was very easy to use and it stayed in her hair overnight and the whole next day Easy to remove from hair\n\nMy two daughters absolutely love glitter or anything shiny so the Blinger Ultimate Set was a perfect gift for them It is very easy for them to use with younger kids and adult should help read instructions but after that they will get it\n\nMy daughter loves this Such a fun gift We have blinged her hair every day since Christmas Easy to remove the gems at the end of the day and they stay nicely in her hair all day too\n\nPerfect gift for my 6 year old granddaughter Im sure she will love it for a few years She blings everything\n\nIt was a great deal and my daughter gets so much joy out of using it\n\nI liked using in my hair totally fun on special occasions\n\nVery disappointed in what was advertised and what I received Advertised as receiving in a big box and with 225 jewels Received in a tiny box with about 12 jewels as ad

In [22]:
memory.get_relevant("daughter", num_relevant=1)

['Stuck to 45 year olds inside of ear We tried to remove but child was nervous about us touching his ear Went to ER same night and they pushed it deeper Went to ENT and he could not get it out At this point close to our childs ear drum We ended up having to sedate a young child as advised by specialist to remove Terrible time to have to go to the hospital during high cases of COVID Then we had to wait bc of limited availability So this stone was stuck in his ear for a few days causing bleeding and pain I dont advise unless this is for a teenager and no small kids around\n\nMy daughter requested this for her birthday Honestly its just sparkly 3D stickers With a fancy thing to stamp it on but really you could just peel it off with your finger and stick it anywheres adhesive doesnt last long so these dont stick to clothesfabricshoes etc very long It lasts a day and the You have them all over your house  My daughter enjoyed it though but after she used all the stickies in a couple days  it